This Notebook analyzes the .csv file generated by TVM VM profiler.

To profile a TVM VM

```
report = vm.profile("encode", audio_data, *effects, *const_params_dict["encode"])
csv = report.csv()

with open("profile_stream.csv", "w", encoding="utf-8") as f:
    f.write(csv)
    print("Profile saved to profile_stream.csv")
```

In [8]:
import pandas as pd

df = pd.read_csv("profile_stream.csv")

df["Duration (ms)"] = df["Duration (us)"] / 1000
total_time_ms = df["Duration (ms)"].sum()

K = 50

# Get the top K time-consuming operators
top_k_operators = df.nlargest(K, "Duration (ms)")[["Name", "Argument Shapes", "Duration (ms)"]]

# Get only the top K operators with conv1d in it
# top_k_operators = top_k_operators[top_k_operators["Name"].str.contains("conv1d")]

print(f"Total Time (ms): {total_time_ms:.3f}")
print("Top K Time-Consuming Operators:")
print(top_k_operators.to_string(index=False))

Total Time (ms): 43.627
Top K Time-Consuming Operators:
                                   Name                                                                       Argument Shapes  Duration (ms)
           vm.builtin.check_tensor_info                                                                    float32[1, 1, 512]       0.875334
                  fused_conv1d38_add301 float32[1, 512, 16], float32[1024, 512, 16], float32[1, 1024, 1], float32[1, 1024, 1]       0.460000
                                   topk                                                         float32[1, 1024], int32[1, 1]       0.387792
    vm.builtin.cached_padding_1d_update                                                                   float32[1, 64, 512]       0.373417
      fused_reshape16_snake2_reshape161                          float32[1, 256, 64], float32[1, 256, 1], float32[1, 256, 64]       0.364334
    vm.builtin.cached_padding_1d_update                                                           